In [1]:
!pip install numpy mlflow tensorflow "ray[serve,default,client]"

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 242.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 191.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 214.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 235.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 161.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 175.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 133.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 36.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import mlflow
import mlflow.tensorflow
import numpy as np
# import joblib

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

mlflow.set_tracking_uri(uri="http://ai-starter-kit-mlflow:5000")

# -------------------
# Prepare Data
# -------------------
data = load_diabetes()
X = data.data
y = data.target.reshape(-1, 1)

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------
# Define Model
# -------------------
def create_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(input_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)  # regression output
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mse"])
    return model

input_dim = X_train.shape[1]
epochs = 50
batch_size = 32

mlflow.set_experiment("Diabetes_Prediction_TensorFlow")

with mlflow.start_run():
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_fn", "mse")
    mlflow.log_param("input_features", input_dim)

    model = create_model(input_dim)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Evaluation
    loss, mse = model.evaluate(X_test, y_test, verbose=0)
    rmse = np.sqrt(mse)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)

    # # Save scaler for serving
    # joblib.dump(scaler, "scaler.pkl")
    # mlflow.log_artifact("scaler.pkl")

    # # Log TensorFlow model
    # mlflow.tensorflow.log_model(model, "tf_model")


2025/08/22 09:44:41 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1001'
/tmp/models-cache/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🏃 View run enchanting-sloth-205 at: http://ai-starter-kit-mlflow:5000/#/experiments/1/runs/696471a5d370436194a52e891e52694c
🧪 View experiment at: http://ai-starter-kit-mlflow:5000/#/experiments/1


In [10]:
# import joblib
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
from starlette.requests import Request
from typing import Dict

from ray import serve
import ray

# Connect to your Ray cluster
# ray.init("ray://ai-starter-kit-kuberay-head-svc:10001")

# Change this to your actual MLflow run path or model registry URI
# Example for a run artifact:
# MLFLOW_MODEL_URI = "mlruns/0/<RUN_ID>/artifacts/tf_model"
# Example for a registered model:
# MLFLOW_MODEL_URI = "models:/Diabetes_Prediction_TensorFlow/1"
# MLFLOW_MODEL_URI = "mlruns/0/<RUN_ID>/artifacts/tf_model"


@serve.deployment(
    ray_actor_options={
        "runtime_env": {
            "pip": ["tensorflow"]
        },
    }
)
class TensorFlowMLflowDeployment:
    def __init__(self):
        print("Loading model from MLflow...")
        self.model = model
        print("Model loaded successfully.")

        # print("Loading scaler...")
        # self.scaler = joblib.load("scaler.pkl")  # can also be downloaded from MLflow artifacts
        # print("Scaler loaded successfully.")

    async def __call__(self, request: Request) -> Dict:
        try:
            data = await request.json()
            features = data.get("features", None)
            if features is None:
                return {"error": "Missing 'features' in request"}

            # Scale features
            X = np.array(features).reshape(1, -1)
            X_scaled = X
            # X_scaled = self.scaler.transform(X)

            # Make prediction with TensorFlow model
            prediction = self.model.predict(X_scaled).flatten().tolist()

            return {"prediction": prediction}
        except Exception as e:
            return {"error": str(e)}


# Bind and deploy
app = TensorFlowMLflowDeployment.bind()
serve.run(app, route_prefix="/predict")


INFO 2025-08-22 09:45:07,093 serve 126 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.


RayTaskError(ModuleNotFoundError): [36mray::ServeController.deploy_applications()[39m (pid=3329, ip=10.118.0.131, actor_id=1391147c7fbc19479214196e01000000, repr=<ray.serve._private.controller.ServeController object at 0x79cf403b0ce0>)
  File "/home/ray/anaconda3/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/home/ray/anaconda3/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
           ^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ray/anaconda3/lib/python3.12/site-packages/ray/serve/_private/controller.py", line 786, in deploy_applications
    self.application_state_manager.deploy_apps(name_to_deployment_args)
  File "/home/ray/anaconda3/lib/python3.12/site-packages/ray/serve/_private/application_state.py", line 1001, in deploy_apps
    self._application_states[name].d